<a href="https://colab.research.google.com/github/Suzaq2005/Sentimental-Analysis/blob/main/maze_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import random


In [33]:
# Directions the agent can face
DIRECTIONS = ['NORTH', 'EAST', 'SOUTH', 'WEST']

In [34]:
# Movement mappings for directions (delta row, delta col)
MOVES = {
    'NORTH': (-1, 0),
    'EAST': (0, 1),
    'SOUTH': (1, 0),
    'WEST': (0, -1)
}

In [35]:
# Sample maze (0 = open path, 1 = wall, 2 = exit)
maze = [
    [0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0],
    [0, 0, 0, 1, 0],
    [1, 0, 0, 0, 0],
    [0, 0, 0, 1, 2]
]

In [36]:



class ImprovedReactiveMazeAgent:
    def __init__(self, maze, start):
        self.maze = maze
        self.position = start  # (row, col)
        self.direction = 'NORTH'  # Initially facing north
        self.visited = set()  # Keep track of visited cells
        self.path_memory = []  # Memory of the path taken for backtracking
        self.unvisited_stack = []  # Stack to keep track of unvisited cells

    def move(self):
        # Apply the behavioral rules to decide the next move

        # Check for adjacent walls
        left_dir = self.get_left_direction()
        front_dir = self.direction
        right_dir = self.get_right_direction()

        # Prioritize unvisited cells if available
        next_move = None
        if not self.is_wall(self.look(left_dir)) and self.look(left_dir) not in self.visited:
            next_move = left_dir
        elif not self.is_wall(self.look(front_dir)) and self.look(front_dir) not in self.visited:
            next_move = front_dir
        elif not self.is_wall(self.look(right_dir)) and self.look(right_dir) not in self.visited:
            next_move = right_dir

        if next_move:
            # Follow unvisited paths
            if next_move == left_dir:
                self.turn_left()
            elif next_move == right_dir:
                self.turn_right()
            self.go_forward()
        else:
            # Backtrack if no new path available
            self.retrace_steps()

        # Update memory and visited positions
        self.visited.add(self.position)
        self.path_memory.append(self.position)

    def look(self, direction):
        """Look in a specific direction and return the cell in that direction."""
        row, col = self.position
        delta_row, delta_col = MOVES[direction]
        return (row + delta_row, col + delta_col)

    def is_wall(self, position):
        """Check if the given position is a wall or out of bounds."""
        row, col = position
        if row < 0 or col < 0 or row >= len(self.maze) or col >= len(self.maze[0]):
            return True
        return self.maze[row][col] == 1

    def go_forward(self):
        """Move forward in the current direction."""
        next_position = self.look(self.direction)
        if not self.is_wall(next_position):
            self.position = next_position

    def turn_left(self):
        """Turn left relative to current direction."""
        current_index = DIRECTIONS.index(self.direction)
        self.direction = DIRECTIONS[(current_index - 1) % 4]

    def turn_right(self):
        """Turn right relative to current direction."""
        current_index = DIRECTIONS.index(self.direction)
        self.direction = DIRECTIONS[(current_index + 1) % 4]

    def get_left_direction(self):
        """Get the direction to the left relative to current direction."""
        current_index = DIRECTIONS.index(self.direction)
        return DIRECTIONS[(current_index - 1) % 4]

    def get_right_direction(self):
        """Get the direction to the right relative to current direction."""
        current_index = DIRECTIONS.index(self.direction)
        return DIRECTIONS[(current_index + 1) % 4]

    def retrace_steps(self):
        """Retrace steps towards unexplored areas."""
        if len(self.path_memory) > 1:
            self.path_memory.pop()  # Remove current position from memory
            self.position = self.path_memory[-1]  # Move to the previous position
        else:
            pass  # No more steps to retrace (edge case)

    def detect_dynamic_changes(self):
        """Detect and respond to changes in the environment."""
        adjacent_positions = [self.look(dir) for dir in DIRECTIONS]
        for pos in adjacent_positions:
            if self.is_wall(pos) and pos in self.visited:
                # Mark as dead-end if the path has become a wall
                pass

    def at_exit(self):
        """Check if the agent has reached the exit."""
        return self.maze[self.position[0]][self.position[1]] == 2

    def run(self):
        """Run the agent until it finds the exit or gets stuck."""
        steps = 0  # Count the number of moves
        while not self.at_exit():
            self.move()
            self.detect_dynamic_changes()  # Check for dynamic changes each move
            steps += 1
            if steps > 1000:  # Fail-safe in case the agent gets stuck in an infinite loop
                print("Agent got stuck after 1000 moves")
                break
        return self.position, steps

# Create the agent and run with the new strategy
improved_agent = ImprovedReactiveMazeAgent(maze, start=(0, 0))
result_position, steps_taken = improved_agent.run()
result_position, steps_taken


((4, 4), 8)